## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
# Input info
se_cells_meta_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/se_cells_meta.tsv"
p_thresh = 0.1 
out_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/hypergeom_clone_clust/mincl.10_bothConds.False/hypergeom.csv" 

min_clone_size = 10
conds_sep = False

In [2]:
import pandas as pd
import numpy as np
from os.path import join
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

In [4]:
cells_meta = pd.read_csv(se_cells_meta_f, sep="\t")
cells_meta = cells_meta.loc[~(cells_meta["name"]=="None")]
sizes = cells_meta.groupby("name").size().sort_values(ascending=False)

if conds_sep:
    name_cond_size = cells_meta.groupby(["name","condition"]).size()
    name_cond_size = name_cond_size[name_cond_size>min_clone_size].reset_index()
    tmp = name_cond_size.groupby("name").size()>1
    clones_filt = tmp[tmp==True].index #sizes.loc[tmp[tmp==True].index]

else:
    name_cond_size = cells_meta.groupby(["name"]).size()
    name_cond_size = name_cond_size[name_cond_size>min_clone_size]
    clones_filt = name_cond_size.index 

sizes = sizes.loc[clones_filt].sort_values(ascending=False)
clones_filt

groups = cells_meta.groupby(["seurat_clusters", "name"]).size().reset_index().rename({0:"count"}, axis=1)
groups

clones = clones_filt#np.unique(groups["name"])
clones

atac_cl = np.unique(groups["seurat_clusters"])
atac_cl

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [5]:
#p(k,M,n,N) = (n choose k)((M-n)choose(N-k))/(MchooseN)
#pmf(k, M, n, N) = choose(n, k) * choose(M - n, N - k) / choose(M, N),
                               #for max(0, N - (M-n)) <= k <= min(n, N)
    
# M: Total number of cells
# n: Number of cells in the atac cluster (group population)
# N: Number of cells in clone (the draw)
# x: Number of cells in specific clone and cluster
enrichment_df = pd.DataFrame(index=clones,
                             columns=atac_cl, dtype=np.float128)

M = groups["count"].sum()
for cl in clones:
    for atac in atac_cl:     
        n = groups[groups["seurat_clusters"]==atac]["count"].sum()
        N = groups[groups["name"]==cl]["count"].sum()
        
        x = groups[((groups["name"]==cl)& (groups["seurat_clusters"]==atac))]["count"].sum()
        
        # rv = hypergeom(M, n, N)
        prb = 1 - hypergeom.cdf(x, M, n, N)
        enrichment_df.loc[cl, atac] = prb
        

In [6]:
enrichment_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
name,,,,,,,,,,,,,,,,
0_0,1.000000e+00,1.000000e+00,0.000000,0.999239,9.567640e-07,1.000000e+00,1.910039e-10,0.003207,1.000000e+00,3.421863e-11,0.999736,0.274550,0.497786,0.678001,0.594063,0.042991
0_1,1.000000e+00,9.999991e-01,0.000000,0.999813,1.585660e-06,1.000000e+00,2.625352e-05,0.061357,1.000000e+00,6.872930e-03,0.994344,0.441781,0.072279,0.462871,0.869576,0.217004
0_10,1.000000e+00,9.999996e-01,0.001486,0.997997,1.049942e-08,9.999945e-01,2.557463e-08,0.156961,9.999988e-01,3.837998e-05,0.992367,0.040294,0.307919,0.322645,0.728337,0.107912
0_11,9.999998e-01,9.999962e-01,0.001833,0.988723,4.052073e-05,9.999992e-01,4.196887e-08,0.000135,9.999850e-01,1.637007e-02,0.800671,0.262218,0.294788,0.535043,0.720811,0.017368
0_12,1.000000e+00,9.986781e-01,0.000004,0.985615,3.979696e-03,9.999999e-01,3.051857e-06,0.000071,9.998519e-01,1.197179e-03,0.887967,0.879436,0.494134,0.285670,0.707141,0.097779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1_5,2.451358e-04,2.960403e-08,1.000000,0.024792,9.990799e-01,1.278135e-04,1.000000e+00,0.957825,2.660644e-01,9.999867e-01,0.000004,0.104248,0.797121,0.142579,0.860286,0.206283
1_6,7.465733e-07,2.206291e-04,1.000000,0.201747,9.999998e-01,1.637492e-05,1.000000e+00,0.999908,5.808476e-08,9.999940e-01,0.018273,0.877509,0.370938,0.116948,0.002477,0.190331
1_7,9.860603e-09,2.845365e-03,1.000000,0.002139,9.999906e-01,1.745461e-03,1.000000e+00,0.986833,8.602935e-06,1.000000e+00,0.012244,0.732705,0.038679,0.753657,0.831802,0.177708


In [7]:
reject, pvals_corrected, _, _ = multitest.multipletests(enrichment_df.values.flatten(), alpha=p_thresh, method="fdr_bh")

In [8]:
nrows, ncols = enrichment_df.shape
reject, pvals_corrected, _, _ = multitest.multipletests(enrichment_df.values.flatten(),
                                                        alpha=p_thresh, method="fdr_bh")
pvals_corrected = np.reshape(pvals_corrected, [nrows,ncols])
bh_enrichment_df = enrichment_df.copy()
bh_enrichment_df.loc[:,:] = pvals_corrected
bh_enrichment_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
name,,,,,,,,,,,,,,,,
0_0,1.000000e+00,1.000000,0.000000,1.000000,2.128800e-05,1.000000e+00,1.942782e-08,0.017840,1.000000,4.429757e-09,1.000000,0.612789,0.900695,1.000000,1.000000,0.149317
0_1,1.000000e+00,1.000000,0.000000,1.000000,3.225685e-05,1.000000e+00,3.337947e-04,0.199937,1.000000,3.340291e-02,1.000000,0.833240,0.227711,0.858114,1.000000,0.520224
0_10,1.000000e+00,1.000000,0.009575,1.000000,5.339708e-07,1.000000e+00,1.040522e-06,0.413913,1.000000,4.516784e-04,1.000000,0.142027,0.661352,0.682684,1.000000,0.308937
0_11,1.000000e+00,1.000000,0.011111,1.000000,4.653349e-04,1.000000e+00,1.451052e-06,0.001246,1.000000,6.937792e-02,1.000000,0.595531,0.641888,0.939459,1.000000,0.072528
0_12,1.000000e+00,1.000000,0.000071,1.000000,2.083488e-02,1.000000e+00,5.718217e-05,0.000751,1.000000,7.929220e-03,1.000000,1.000000,0.896365,0.630431,1.000000,0.289469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1_5,2.006169e-03,0.000001,1.000000,0.097526,1.000000e+00,1.193081e-03,1.000000e+00,1.000000,0.600437,1.000000e+00,0.000076,0.305079,1.000000,0.385261,1.000000,0.505589
1_6,1.771867e-05,0.001870,1.000000,0.496178,1.000000e+00,2.263872e-04,1.000000e+00,1.000000,0.000002,1.000000e+00,0.075206,1.000000,0.742927,0.327824,0.014348,0.474620
1_7,5.339708e-07,0.016207,1.000000,0.012693,1.000000e+00,1.075990e-02,1.000000e+00,1.000000,0.000129,1.000000e+00,0.054315,1.000000,0.138391,1.000000,1.000000,0.453505


In [9]:
output_df = pd.DataFrame(index=sizes.index)
output_df["significant clusters"] = ""
output_df["size"] = sizes
#output_df["min_significance"] = None

sig_results = []
sig_order = []
for ind, val in bh_enrichment_df.loc[sizes.index].iterrows():
    passed = val[val<p_thresh].index.values
    if len(passed)>0:
        output_df.loc[ind, "significant clusters"] = ";".join([str(x) for x in passed])
        output_df.loc[ind, "min_significance"] = min(val)
       # sig_results.append((ind, passed))
output_df.loc[:, bh_enrichment_df.columns] = bh_enrichment_df.loc[output_df.index]
output_df = output_df.sort_values("min_significance")
output_df


,significant clusters,size,min_significance,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
name,,,,,,,,,,,,,,,,,,,
0_0,2;4;6;7;9,455,0.000000,1.000000,1.000000,0.000000,1.000000,0.000021,1.000000,1.942782e-08,0.017840,1.000000,4.429757e-09,1.000000,0.612789,0.900695,1.000000,1.000000,0.149317
1_0,0;1;5;8,449,0.000000,0.000002,0.000005,1.000000,0.236383,1.000000,0.000001,1.000000e+00,1.000000,0.000000,1.000000e+00,1.000000,1.000000,1.000000,0.545115,0.466665,1.000000
0_1,2;4;6;9,299,0.000000,1.000000,1.000000,0.000000,1.000000,0.000032,1.000000,3.337947e-04,0.199937,1.000000,3.340291e-02,1.000000,0.833240,0.227711,0.858114,1.000000,0.520224
0_4,2;4;6;7;9,239,0.000000,1.000000,1.000000,0.000000,1.000000,0.004666,1.000000,3.151140e-06,0.007896,1.000000,3.058834e-02,1.000000,0.704233,0.609011,0.397996,0.879569,0.905600
0_23,2;4;6;9,141,0.000000,1.000000,1.000000,0.000000,1.000000,0.011104,1.000000,6.690157e-02,0.121977,1.000000,6.333162e-02,1.000000,0.958607,1.000000,1.000000,1.000000,0.704233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1_35,1;3;10,114,0.004022,0.179074,0.004022,1.000000,0.019332,1.000000,0.203538,1.000000e+00,1.000000,0.252721,1.000000e+00,0.081191,0.755305,1.000000,0.908965,0.145050,0.628712
1_41,0;1;8,67,0.006332,0.007506,0.012423,1.000000,0.216491,1.000000,0.515224,1.000000e+00,1.000000,0.006332,1.000000e+00,0.805730,1.000000,0.587979,0.241697,0.736603,0.454037
0_44,4;7,14,0.007832,1.000000,0.860376,0.288840,0.854127,0.007832,1.000000,7.013961e-01,0.044153,1.000000,5.696899e-01,0.770653,0.683353,0.468231,0.474620,0.270752,0.141754


In [10]:
output_df = output_df.loc[~(output_df["min_significance"].isnull())]

In [11]:
output_df.to_csv(out_f, sep=",")